In [2]:
import d2lzh as d2l
import mxnet as mx
from mxnet import gluon, init
from mxnet.gluon import loss as gloss
from mxnet import autograd, nd
from mxnet import nd
import time
import matplotlib.pyplot as plt
import numpy as np
from mxnet.io import ImageRecordIter
import os, time, shutil 
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms
from gluoncv.utils import makedirs
from gluoncv.model_zoo import get_model
from mxnet.gluon import data as gdata, loss as gloss, utils as gutils
import sys


ModuleNotFoundError: No module named 'mxnet'

# The structure of the modified mobilenet

In [2]:
model_name = 'mobilenetv3_small'
classes = 5
finetune_net = get_model(model_name, pretrained=True)

def conv_block(num_channels):
    blk = nn.HybridSequential() 
    blk.add(nn.Conv2D(num_channels,(1,1)),
            nn.BatchNorm(),
            nn.Activation('relu'),
            nn.GlobalAvgPool2D())
    return blk

class modifiedMobileNet(nn.HybridBlock):
  
    def __init__(self, **kwargs):
        super(modifiedMobileNet, self).__init__(**kwargs)
        finetune_net = get_model(model_name, pretrained=True)
        with self.name_scope():
          self.part1 = finetune_net.features[:3]
          self.part2 = finetune_net.features[3]
          self.part3 = finetune_net.features[4]
          self.part4 = finetune_net.features[5]
          self.part5 = finetune_net.features[6:21]
          self.part6 = nn.Dense(classes)
         # self.part6 = nn.Conv2D(classes, kernel_size=(1, 1))
          self.flatten = nn.Flatten()
          self.dropout = nn.Dropout(0.5)
          self.conv1 = conv_block(64)
          self.conv2 = conv_block(64)
          self.conv3 = conv_block(64)
          self.conv4 = conv_block(64)
          self.conv5 = conv_block(64)
          self.w = conv_block(5)
     

    def forward(self,x):
            w = self.w(x)
            w = self.flatten(w)
       #     print(w.shape)
            x = self.part1(x)
            x1 = self.conv1(x)
       #     print(x1.shape)
            x = self.part2(x)
            x2 = self.conv2(x)
       #     print(x2.shape)
            x = self.part3(x)
            x3 = self.conv3(x)
      #      print(x3.shape)
            x = self.part4(x)
            x4 = self.conv4(x)
      #      print(x4.shape)
            x = self.part5(x)
            x5 = self.conv5(x)
      #      print(x5.shape)
            x = w[0]*x1+w[1]*x2+w[2]*x3+w[3]*x4+w[4]*x5
          #  x = nd.concat(x1, x2, x3, x4, x5, dim = 1)
            x = self.dropout(x)
            x = self.part6(x)
            return x

# functions used in training:

In [3]:
#  The following functions referred to the code provided in  https://d2l.ai/

def try_all_gpus():  
    ctxes = []
    try:
        for i in range(16):  
            ctx = mx.gpu(i)
            _ = nd.array([0], ctx=ctx)
            ctxes.append(ctx)
    except mx.base.MXNetError:
        pass
    if not ctxes:
        ctxes = [mx.cpu()]
    return ctxes

ctx = try_all_gpus()

def _get_batch(batch, ctx):
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])
    
def evaluate_accuracy(data_iter, net, ctx=[mx.cpu()]):
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n = nd.array([0]), 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            y = y.astype('float32')
            acc_sum += (net(X).argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n

def train(train_iter, test_iter, net, loss, trainer, ctx, num_epochs):
    print('training on', ctx)
    global hold_accuracy_train
    global hold_accuracy_val
    hold_accuracy_train = []    
    hold_accuracy_val = []
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, m, start = 0.0, 0.0, 0, 0, time.time()
        for i, batch in enumerate(train_iter):
            Xs, ys, batch_size = _get_batch(batch, ctx)
            with autograd.record():
                y_hats = [net(X) for X in Xs]
                ls = [loss(y_hat, y) for y_hat, y in zip(y_hats, ys)]
            for l in ls:
                l.backward()
            trainer.step(batch_size)
            train_l_sum += sum([l.sum().asscalar() for l in ls])
            n += sum([l.size for l in ls])
            train_acc_sum += sum([(y_hat.argmax(axis=1) == y).sum().asscalar()
                                 for y_hat, y in zip(y_hats, ys)])
            m += sum([y.size for y in ys])
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        hold_accuracy_val.append(test_acc)
     #   train_acc = evaluate_accuracy(train_iter, net, ctx)
        hold_accuracy_train.append(train_acc_sum / m)
        print('epoch %d, loss %.4f, train acc %.3f, val acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / m, test_acc,
                 time.time() - start))

In [4]:
class DataIterLoader():
    def __init__(self, data_iter):
        self.data_iter = data_iter

    def __iter__(self):
        self.data_iter.reset()
        return self

    def __next__(self):
        batch = self.data_iter.__next__()
        assert len(batch.data) == len(batch.label) == 1
        data = batch.data[0]
        label = batch.label[0]
        return data, label


In [16]:
def get_score(round, model, train_iter, val_iter, test_iter):
    ctx = try_all_gpus()
    loss = gloss.SoftmaxCrossEntropyLoss()
    net1 = model     
    net1.collect_params('.*modifiedmobilenet').initialize(init.Xavier(), ctx = ctx)
    net1.collect_params().reset_ctx(ctx)
    net1.hybridize()
    epochs = 20
    trainer1 = gluon.Trainer(net1.collect_params(), 'adam') #{'learning_rate': 0.1, 'wd': 1e-6}
    train(train_iter, val_iter, net1, loss, trainer1, ctx, num_epochs=epochs)
    final_test_acc = evaluate_accuracy(test_iter, net1, ctx)
    print('modified_mobilenetv3_small:', final_test_acc)

    epochs = 20
    trainer2 = gluon.Trainer(net1.collect_params(), 'sgd') #{'learning_rate': 0.1, 'wd': 1e-6}
    train(train_iter, val_iter, net1, loss, trainer2, ctx, num_epochs=epochs)
    final_test_acc = evaluate_accuracy(test_iter, net1, ctx)
    print('modified_mobilenetv3_small:', final_test_acc)
    file_name = 'k_fold_cross_validation/models/net_'+str(round)+'.params'
    net1.save_parameters(file_name)
    return evaluate_accuracy(val_iter, net1, ctx)


In [17]:
result = []
for i in range(5):
    rec_path = os.path.expanduser('k_fold_cross_validation/rec/')

    # You need to specify ``root`` for ImageNet if you extracted the images into
    # a different folder
    train_data = ImageRecordIter(
        path_imgrec = os.path.join(rec_path, 'Train_rec_'+str(i)+'.rec'),
        path_imgidx = os.path.join(rec_path, 'Train_rec_'+str(i)+'.idx'),
        resize = 299,
        data_shape  = (3, 299, 299),
        batch_size  = 128,   
        preprocess_threads = 8,
        shuffle = True,
    )

    val_data = ImageRecordIter(
        path_imgrec = os.path.join(rec_path, 'Val_rec_'+str(i)+'.rec'),
        path_imgidx = os.path.join(rec_path, 'Val_rec_'+str(i)+'.idx'),
        resize = 299,
        data_shape  = (3, 299, 299),
        batch_size  = 64, 
        preprocess_threads = 8,
        shuffle = False
    )

    test_data = ImageRecordIter(
        path_imgrec = os.path.join(rec_path, 'Test_rec.rec'),
        path_imgidx = os.path.join(rec_path, 'Test_rec.idx'),
        resize = 299,
        data_shape  = (3, 299, 299),
        batch_size  = 100, 
        preprocess_threads = 8,
        shuffle = False,
    )

    train_iter = DataIterLoader(train_data)
    val_iter = DataIterLoader(val_data)
    test_iter = DataIterLoader(test_data)

    acc = get_score(i, modifiedMobileNet(), train_iter, val_iter, test_iter)
    result.append(acc)

training on [gpu(0), gpu(1), gpu(2), gpu(3)]
epoch 1, loss 0.8015, train acc 0.731, val acc 0.950, time 44.3 sec
epoch 2, loss 0.2999, train acc 0.910, val acc 0.970, time 39.6 sec
epoch 3, loss 0.2004, train acc 0.946, val acc 0.963, time 38.6 sec
epoch 4, loss 0.1378, train acc 0.962, val acc 0.980, time 39.3 sec
epoch 5, loss 0.0998, train acc 0.976, val acc 0.982, time 39.0 sec
epoch 6, loss 0.0970, train acc 0.976, val acc 0.984, time 39.2 sec
epoch 7, loss 0.0749, train acc 0.979, val acc 0.981, time 39.0 sec
epoch 8, loss 0.0635, train acc 0.983, val acc 0.979, time 38.9 sec
epoch 9, loss 0.0501, train acc 0.985, val acc 0.972, time 39.2 sec
epoch 10, loss 0.0571, train acc 0.986, val acc 0.981, time 39.3 sec
epoch 11, loss 0.0486, train acc 0.986, val acc 0.934, time 39.6 sec
epoch 12, loss 0.0449, train acc 0.989, val acc 0.979, time 38.6 sec
epoch 13, loss 0.0406, train acc 0.990, val acc 0.958, time 39.0 sec
epoch 14, loss 0.0389, train acc 0.991, val acc 0.984, time 38.9 se

epoch 15, loss 0.0068, train acc 0.998, val acc 0.988, time 40.2 sec
epoch 16, loss 0.0084, train acc 0.998, val acc 0.987, time 38.9 sec
epoch 17, loss 0.0075, train acc 0.998, val acc 0.990, time 39.2 sec
epoch 18, loss 0.0072, train acc 0.998, val acc 0.987, time 39.4 sec
epoch 19, loss 0.0045, train acc 0.999, val acc 0.990, time 39.1 sec
epoch 20, loss 0.0086, train acc 0.998, val acc 0.988, time 40.3 sec
modified_mobilenetv3_small: 0.992
training on [gpu(0), gpu(1), gpu(2), gpu(3)]
epoch 1, loss 0.8097, train acc 0.731, val acc 0.949, time 97.3 sec
epoch 2, loss 0.3298, train acc 0.909, val acc 0.958, time 39.3 sec
epoch 3, loss 0.2008, train acc 0.946, val acc 0.970, time 38.5 sec
epoch 4, loss 0.1242, train acc 0.967, val acc 0.972, time 38.9 sec
epoch 5, loss 0.1015, train acc 0.969, val acc 0.959, time 39.8 sec
epoch 6, loss 0.0766, train acc 0.979, val acc 0.981, time 39.2 sec
epoch 7, loss 0.0762, train acc 0.980, val acc 0.974, time 38.9 sec
epoch 8, loss 0.0720, train acc

# Original MobileNet

In [5]:
def get_score(round, train_iter, val_iter, test_iter):
    model_name = 'mobilenetv3_small'
    classes = 5
    finetune_net = get_model(model_name, pretrained=True)
    output = nn.HybridSequential()
    output.add(nn.Conv2D(classes, (1,1)))
    output.add(nn.Flatten())
    with finetune_net.name_scope():
        finetune_net.output = output
    ctx = try_all_gpus()
    loss = gloss.SoftmaxCrossEntropyLoss()


    finetune_net.output.initialize(init.Xavier(), ctx = ctx)
    finetune_net.collect_params().reset_ctx(ctx)
    finetune_net.hybridize()
    epochs = 20
    trainer1 = gluon.Trainer(finetune_net.collect_params(), 'adam') #{'learning_rate': 0.1, 'wd': 1e-6}
    train(train_iter, val_iter, finetune_net, loss, trainer1, ctx, num_epochs=epochs)
    final_test_acc = evaluate_accuracy(test_iter, finetune_net, ctx)
    print('original_mobilenetv3_small:', final_test_acc)

    epochs = 20
    trainer2 = gluon.Trainer(finetune_net.collect_params(), 'sgd') #{'learning_rate': 0.1, 'wd': 1e-6}
    train(train_iter, val_iter, finetune_net, loss, trainer2, ctx, num_epochs=epochs)
    final_test_acc = evaluate_accuracy(test_iter, finetune_net, ctx)
    print('original_mobilenetv3_small:', final_test_acc)
    file_name = 'k_fold_cross_validation_3/models/original_'+str(round)+'.params'
    finetune_net.save_parameters(file_name)
    return evaluate_accuracy(test_iter, finetune_net, ctx)


In [ ]:
result = []
#os.mkdir('k_fold_cross_validation_4/models')
for i in range(5):
    rec_path = os.path.expanduser('k_fold_cross_validation/rec/')

    # You need to specify ``root`` for ImageNet if you extracted the images into
    # a different folder
    train_data = ImageRecordIter(
        path_imgrec = os.path.join(rec_path, 'Train_rec_'+str(i)+'.rec'),
        path_imgidx = os.path.join(rec_path, 'Train_rec_'+str(i)+'.idx'),
        resize = 299,
        data_shape  = (3, 299, 299),
        batch_size  = 128,   
        preprocess_threads = 8,
        shuffle = True,
    )

    val_data = ImageRecordIter(
        path_imgrec = os.path.join(rec_path, 'Val_rec_'+str(i)+'.rec'),
        path_imgidx = os.path.join(rec_path, 'Val_rec_'+str(i)+'.idx'),
        resize = 299,
        data_shape  = (3, 299, 299),
        batch_size  = 64, 
        preprocess_threads = 8,
        shuffle = False
    )

    test_data = ImageRecordIter(
        path_imgrec = os.path.join(rec_path, 'Test_rec.rec'),
        path_imgidx = os.path.join(rec_path, 'Test_rec.idx'),
        resize = 299,
        data_shape  = (3, 299, 299),
        batch_size  = 100, 
        preprocess_threads = 8,
        shuffle = False,
    )

    train_iter = DataIterLoader(train_data)
    val_iter = DataIterLoader(val_data)
    test_iter = DataIterLoader(test_data)

    acc = get_score(i, train_iter, val_iter, test_iter)
    result.append(acc)
print(result)